# Food Finder w/ Google APi
#### Using Google API Here is the testing grounds of working with our python script
#### Here we create a google api client to pull requests we willl use this class client to operate more on our calls

# Goals
 - Enrich a pandas with information using class "client"
 - Automate more of the client via automating end points since it repeats
 - Set up for user input such as "location queary"

In [2]:
import googlemaps #Uses google api
import env

#import pandas as pd (For dataframe use for saving favorite spots?)

#encoding url
from urllib.parse import urlencode
from pandas.core import base
#requests to server?
import requests

In [17]:
class GoogleMapsClient(object):
    lat = None,
    lng = None

    data_type = "json"
    location_query = None
    api_key = None 

    def __init__(self , api_key = None, address_or_postal_code = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception("API KEY IS REQUIRED!")
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_long()



    # Gets Lat and long
    def extract_lat_long(self):
        location_query = self.location_query
        #if location != None:
        #    location_query = location

        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {"address": location_query, "key": self.api_key}

        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"

        req = requests.get(url)
        if req.status_code not in range(200,299):
            return {}
        lat_long = {}
        try:
            lat_long = req.json()['results'][0]['geometry']['location']
        except:
            pass
        lat, lng = lat_long.get('lat'), lat_long.get('lng')
        self.lat = lat
        self.lng = lng
        return lat, lng

    # Gets list of locations nearby
    def search(self, keyword= "Mexican food", location = None, radius = 1000):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_long(location = location)
        endpoint = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}"
        params = {"key": self.api_key,
            "location": f"{lat}, {lng}",
            "radius": radius,
            "keyword" : keyword
        }
        params_encoded = urlencode(params)

        places_url = f"{endpoint}?{params_encoded}"

        r = requests.get(places_url)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

    # Gets detail locations from place (name, rating, phone number, and address)
    # need a location Id
    def deatil(self, place_id = "", fields = ["name", "rating", "formatted_phone", "formatted_address"]):
        detail_base_endPoint = "https://maps.googleapis.com/maps/api/place/details/{self.data_type}"
        detail_params = {
            "place_id" : f"{place_id}",
            "fields" : ",".join(fields),
            "key" : self.api_key
        }
        detail_params_encoded = urlencode(detail_params)
        detail_url = f"{detail_base_endPoint}?{detail_params_encoded}"
        r = requests.get(detail_url)
        if r.status_code not in range(200,299):
            return {}
        return r.json()


In [18]:
Google_Api_Key = env.AKI_KEY

In [19]:
client = GoogleMapsClient(api_key = Google_Api_Key, address_or_postal_code= 'San Antonio, TX')
print(client.lat, client.lng)

29.4295234 -98.4976116
